In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests
import time # using sleep cannot avoid captcha
import pandas as pd

In [8]:
#cases = pd.read_csv("data/ccapjdgmtoutputfile_20210706023712.csv")
cases = pd.read_csv("data/ccapoutputfile_20210706023712.csv")
#cases['CaseNumber'][0]
cases

,CaseNumber,County,CountyName,Name,FillingDate,Status,ModifiedDate,ZipCodes
0,2021SC015147,40,Milwaukee,Rosso Rei LLC,2021-06-30,Open,7/1/2021 3:26:14 PM,53210
1,2021SC015145,40,Milwaukee,251 Bradley Place LLC,2021-06-30,Open,7/1/2021 3:22:54 PM,53208
2,2021SC015143,40,Milwaukee,Shirlean Clayton,2021-06-30,Open,7/1/2021 3:20:41 PM,53225
3,2021SC015142,40,Milwaukee,Milwaukee Moderne LLC,2021-06-30,Open,7/1/2021 3:19:44 PM,53203
4,2021SC015068,40,Milwaukee,David Butler,2021-06-30,Open,7/1/2021 2:08:30 PM,53212
...,...,...,...,...,...,...,...,...
1390,2021SC000070,21,Forest,Darrell Ward,2021-06-14,Open,6/22/2021 8:39:42 PM,54566
1391,2021SC000068,50,Price,David Ames,2021-06-04,Closed,7/6/2021 4:59:25 PM,54459
1392,2021SC000051,46,Pepin,John Robelia,2021-06-08,Closed,7/1/2021 4:13:39 PM,54721
1393,2021SC000038,26,Iron,Chadwin Bucknell,2021-06-23,Open,6/23/2021 6:30:14 PM,54525


In [2]:
# r = requests.get("https://wcca.wicourts.gov/case.html")
# r.raise_for_status()
# html = r.text
# doc = BeautifulSoup(html, "html.parser")

In [9]:
options = Options()
options.headless = False
driver = webdriver.Chrome(options=options, executable_path='./chromedriver')
driver.implicitly_wait(1)
driver.get("https://wcca.wicourts.gov/case.html")
type(driver.page_source) # return the full page HTML code

str

### Code below will encounter CAPTCHA for first trial

In [59]:
file_num = 1
case_number = cases['CaseNumber'][0]
county = cases['County'][0]
textbox = driver.find_element_by_name('caseNo').send_keys(case_number)
span = driver.find_element_by_xpath("//span[@class='Select-arrow']").click()
county_choose = driver.find_element_by_xpath("//input[@class='Select-input']").send_keys(cases['CountyName'][0])

finish_choose = driver.find_element_by_xpath("//*[@class='Select-menu-outer']").click()
submit = driver.find_element_by_xpath("//button[@type='submit']").click()
time.sleep(2)
#scrape here

In [60]:
#download the html source file and go back
html_source = driver.page_source
filename = 'case_html/' + str(file_num) + 'case.csv'
with open(filename,'w') as file:
    file.write(html_source)
    file.write('\n')

driver.back()

In [14]:
for i in range(1, 100):
    case_number = cases['CaseNumber'][i]
    county = cases['County'][i]
    textbox = driver.find_element_by_name('caseNo')
    textbox.clear()
    time.sleep(1)
    textbox.send_keys(case_number)
    span = driver.find_element_by_xpath("//span[@class='Select-arrow']").click()
    county_choose = driver.find_element_by_xpath("//input[@class='Select-input']").send_keys(cases['CountyName'][i])

    finish_choose = driver.find_element_by_xpath("//*[@class='Select-menu-outer']").click()
    submit = driver.find_element_by_xpath("//button[@type='submit']").click()
    time.sleep(2)
    
    # scrape the data and store the data here
#     html_source = driver.page_source
#     filename = 'more_case_html/' + str(i+1) + 'case.csv'
#     with open(filename,'w') as file:
#         file.write(html_source)
#         file.write('\n')
    time.sleep(7)
    driver.back()
    time.sleep(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="caseNo"]"}
  (Session info: chrome=95.0.4638.69)


In [20]:
# from IPython.core.display import Image
# driver.save_screenshot("shot1.png")
# Image("shot1.png")